In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# 압축풀기
import zipfile 
with zipfile.ZipFile('/kaggle/input/caterpillar-tube-pricing/data.zip','r') as z:
    z.extractall('data')

In [ ]:
train = pd.read_csv('data/competition_data/train_set.csv')
train.head(10)

In [ ]:
test = pd.read_csv('data/competition_data/test_set.csv')
test.head(10)

In [ ]:
train.shape

In [ ]:
bill = pd.read_csv('data/competition_data/bill_of_materials.csv')
bill.head()
bill.shape

In [ ]:
# 
spec = pd.read_csv('data/competition_data/specs.csv')
spec.head(50)

In [ ]:
# 먼저 해야하는 것들 
# 데이터가 겹치는게 있는지 없는지 우선 확인 -> tube_assembly_id 중복 x 
# 문자로 된 데이터 파악
# 차원 동일 시 해야함 행/열 맞춰주기 

In [ ]:
alldata = pd.concat([train, test])
alldata.shape

In [ ]:
# tube 데이터셋 홈피가 중요하다고 해서 추가 준비/ 확인
tube = pd.read_csv('data/competition_data/tube.csv')
tube.head(10)

# 이전 id는 여러값이 중복으로나와 대표값으로 해야했는데 
alldata = pd.merge(alldata, tube, on='tube_assembly_id', how='left')

In [ ]:
display(tube.shape)
display(bill.shape)
display(tube.shape)

In [ ]:
alldata.shape

In [ ]:
# 문자를 숫자로 바꿔주기 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
alldata['tube_assembly_id'] = le.fit_transform(alldata['tube_assembly_id'])
alldata['supplier'] = le.fit_transform(alldata['supplier'])
alldata['bracket_pricing'] = le.fit_transform(alldata['bracket_pricing'])
alldata.head()

In [ ]:
# 날짜 변수를 추가해 가장 쉽게 점수올리기 
alldata['quote_date'] = alldata['quote_date'].astype('datetime64')
alldata['year'] = alldata['quote_date'].dt.year
alldata.groupby('year')['cost'].mean() # 연도에 따라서 크게크게 편차가 있기떄문에 도움이됨 #1985년이 nan인데, test셋에만 있는 연도라 nan으로 찍힘. train셋에는 1985가 없음

alldata['month'] = alldata['quote_date'].dt.month
alldata['day'] = alldata['quote_date'].dt.day
alldata.head()

# 날짜 변수 추가하면 70정도 오름

In [ ]:
billmi = bill.fillna(-1)
billmi.head()

In [ ]:
# 3/10 

In [ ]:
# 22:46 튜브 데이터 추가후, 문자를 숫자로 변경 
for i in alldata.columns[alldata.dtypes==object]:
    alldata[i] = le.fit_transform(list(alldata[i]))
# '<' not supported between instances of 'str' and 'float' -> 결측치가 있다는 것임 alldata[i] -> list(alldata[i])로 묶어줌 


In [ ]:
# bill도 문자를 숫자로
for i in billmi.columns[billmi.dtypes==object]:
    billmi[i] = le.fit_transform(list(billmi[i]))
# '<' not supported between instances of 'str' and 'float' -> 결측치가 있다는 것임 alldata[i] -> list(alldata[i])로 묶어줌 


In [ ]:
# spec도 문자를 숫자로 
for i in spec.columns[spec.dtypes==object]:
    spec[i] = le.fit_transform(list(spec[i]))


In [ ]:
# billMI 변수 추가 
alldata = pd.merge(alldata, billmi, on='tube_assembly_id', how='left')

In [ ]:
# id는 도움이 되면 안됨. 대회에서는 id에서 뭔가 설명력이 들어가는 경우가 있음. y에 영향을 주는 것이 좀 있음
# 이에 id가 cost에 영향을 주는지 없는지 우선 확인 
# 방법 
# 1) id를 넣고, 안넣고 모델결과로 판단 
# 2) 평가셋을 만들어 id넣고 안넣고 차이
# 3) tree 모델에 넣어 변수 중요도 파악해서 확인 
# 4) 보통 id는 인덱스역할을 하는데 train과 test에 믹싱 id 리스트들이 믹싱되어있으면 도움이 되는 경우가 있음  

In [ ]:
# spec 변수 추가
alldata = pd.merge(alldata, spec, on='tube_assembly_id', how='left')

In [ ]:
alldata

In [ ]:
alldata.head()

In [ ]:
alldata.isnull().sum(1)

In [ ]:
alldata.columns

In [ ]:
from scipy.stats import skew

In [ ]:
alldata.columns

In [ ]:
target = alldata['cost']

In [ ]:
target_log = np.log(target)

In [ ]:
target_log.isnull().sum()

In [ ]:
alldata2 = alldata.drop(['cost','id','quote_date'],axis=1)
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

In [ ]:
test2.columns

In [ ]:
# 그림 - 
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (20, 12))
sns.distplot(train['cost'])

In [ ]:
# 그림 - 
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (20, 12))
sns.distplot(target_log)

In [ ]:
alldata['cost']

In [ ]:
target_log.min()

In [ ]:
# 모델 선택 - 머신러닝 대회에서는 무조건 랜포를 먼저 사용 
# 데이터셋이 단순한지 복잡한지 파악이 안되었기때문에 랜포를 써서 판단

# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(n_jobs=4)
# rf.fit(train2, train['cost'])
# result = rf.predict(test2)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs=4)
rf.fit(train2, target_log)
result = rf.predict(test2)


In [ ]:
target_log

In [ ]:
sub = pd.read_csv('/kaggle/input/caterpillar-tube-pricing/sample_submission.csv')
sub['cost'] = result
sub.head()

In [ ]:
sub.to_csv('class.csv', index = False)

In [ ]:
# 상위 20% 달성하기 
# 데이터셋 추가 -> bill_of_materials.csv 추가 시 불편한게 있어서 이를 해결한 뒤 
# 이 대회에서 도움이 되는 하나가 있는 데 이를 파악하고(y값에 log를 해줘야함. 회귀문제는 무조건 그래프를 그려줘서, 한쪽으로 쏠려있으면 해결 ) 이 다음에 모델 변경 
